In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

# TODO: auto sim cutoff needed
decomp = cluster_model.Decomposer(model, ds, labs)
decomp.load()

Creating decomposer with parameter hash a9ba5d753fd502dc8bbf7b6ccce8ebfe0455dfd8
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Loading dataset from cache
Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.3.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.5.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [3]:
decomp.lattice_scores[0].shape

(512, 512)

In [4]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 1):
    print(lattice[0].sum())
    r = [(layer > related_cutoff) * layer for layer in lattice]
    print(r[0].sum())
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 10):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice = cutoff_lattice(decomp.lattice_scores, related_cutoff=15)

# utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

53.49966726459392
0.0


In [5]:
[d.min() for d in decomp.lattice_scores]
[d.max() for d in decomp.lattice_scores]

[0.9999999999999999,
 0.9999999999999999,
 0.9999999999999999,
 0.9999999999999999,
 0.9999999999999999]

## Isolate Specific Neurons

In [23]:
avoid_set = {
	0: [144, 416],
	# 1: [159],
	# 2: [1],
}

In [32]:
import os
import cluster_model
importlib.reload(cluster_model)

LAYER = 0
NODE = 69
# TODO: this ain't right!! We need specific paths!!
ret = cluster_model.restrict_to_related_path(decomp.lattice_scores, LAYER, NODE,
											avoid_set=avoid_set,
                                              n_paths=10_000)
# Write to file
with open("/tmp/related_paths.txt", "w") as f:
    f.write("\n".join([str(r) for r in ret]))
len(ret)

10000

## Try different scoring method

In [38]:
# TODO: scoring and testing the paths

importlib.reload(cluster_model)

score_path = [69, 1, 107, 289, 97, 508]
# TODO: embeds to here...
to_score = ds
to_score_adj = [s[:300] for s in to_score][:300]
to_score = to_score_adj
importlib.reload(cluster_model)
# TODO: should we work over log everywhere?
_, top_ds, scores = cluster_model.score_for_neuron(decomp, to_score_adj, LAYER, score_path, top_n=20, weighting_per_layer=[0, 1, 2, 1, 0.5, 0.25])


TOP TOKENS 60 [7844, 7865, 7849, 7838, 7873, 7854, 610, 7882, 3289, 7890, 622, 11994, 17694, 10256, 12006, 3297, 10734, 3281, 9144, 18460, 15745, 7241, 15719, 19139, 20197, 7232, 19117, 5678, 3367, 3306, 18093, 11997, 4962, 1874, 2156, 16427, 17360, 21053, 15727, 20202, 5495, 5517, 7111, 8904, 3116, 2306, 12199, 21944, 19221, 3195, 8724, 11494, 6688, 14698, 13184, 4250, 7477, 12918, 3044, 14168]
TOP SCORE IDX [103, 8, 43, 158, 235, 135, 141, 120, 245, 209, 94, 253, 270, 73, 44, 242, 64, 24, 27, 218, 231, 282, 71, 93, 117, 41, 29, 160, 294, 255, 42, 114, 151, 87, 196, 174, 55, 98, 170, 40, 188]
DS SCORED 41
EMB FOR TOP (6, 60, 512)
GOT EMBEDS (6, 3281, 512)
(3281, 6, 512) [69, 1, 107, 289, 97, 508]


In [39]:
import math
import numpy as np
from IPython.core.display import display, HTML
from circuitsvis.utils.render import render
# def get_renderable_scores(scored_t: List[str], scores: List


scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

# scores_per_token_set = [max(s) for s in scores]
# top_args = [s.argmax() for s in scores]

to_score_top = [to_score[i] for i in top_ds]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score_top]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]

# TODO: WHAT IS HAPPENING WITH NAN?
act_simp = [[[[math.exp(100 * tok)]]
             for tok in s] for s in scores_reord]
# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

/tmp/ipykernel_59834/3347453532.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Try more path like method

In [45]:
"""
Some paths we have seen:
[69, 1, 107, 289, 97, 508]

Something after a semicolon, often in code
"""

# TODO: SIOMETHING WRONG HERE WITH THE PATHS
import numpy as np
importlib.reload(cluster_model)
score_path = [69, 1, 107, 289, 97, 508]

"""
Paths which seem to have meaning:

"""

weighting_per_layer = np.ones(len(decomp.lattice_scores) + 1)
# TODO: do we need to weight the layers?

# TODO: BY NEURON WEIGHTING!
weighting_per_layer[LAYER] = 2

for i in range(LAYER):
    weighting_per_layer[i] = 0.0
for i in range(LAYER + 1, len(decomp.lattice_scores)):
    G = 0.8
    # Decrease by Gx per layer
    weighting_per_layer[i] = G ** (i - LAYER - 1)


to_score = [d[:300] for d in ds][:200]
scores = decomp.score(
    to_score,
    score_path=score_path,
    # TODO: how do we weight???
    weighting_per_layer=weighting_per_layer
)

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

# TODO: we have the weights here

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]

GOT EMBEDS (6, 14946, 512)
(14946, 6, 512) [69, 1, 107, 289, 97, 508]


In [44]:
import math
from IPython.core.display import display, HTML
import math
import numpy as np

# TODO: WHAT IS HAPPENING WITH NAN?
act_simp = [[[[(math.exp(tok * 100))]]
             for tok in s] for s in scores_reord]
act_simp = [[[[((tok * 100))]]
             for tok in s] for s in scores_reord]
# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

In [42]:
# to_score_simp = ["Solve x + 5 = m * 33 + 4 *n"]
# scores_simp = decomp.score(
#     to_score_simp,
#     score_path=score_path,
#     # TODO: how do we weight???
#     weighting_per_layer=weighting_per_layer
# )

# scores_per_token_set = np.array([max(s) for s in scores_simp])
# top_args = np.argsort(scores_per_token_set)[::-1]

# tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
#     'input_ids']] for d in to_score]
# tokens_reord = [tokens[i] for i in top_args]
# scores_reord = [scores[i] for i in top_args]
# act_simp = [[[[int(math.exp(tok * 0.001))]]
#              for tok in s] for s in scores_reord]
# # TODO sep fun
# html = render(
#     "TextNeuronActivations",
#     tokens=tokens_reord,
#     activations=act_simp,
#     firstDimensionName="Layer",
#     secondDimensionName="Neuron",
#     firstDimensionLabels=None,
#     secondDimensionLabels=None
# )
# display(HTML(str(html)))